In [1]:
import pandas as pd
import pickle as pkl
from collections import defaultdict
from tqdm import tqdm

In [8]:
!pip install tqdm

In [4]:
pmid_inter_data = pd.read_csv("FinalMappedData_20thMay2020.csv")
nct_pmid_map_data = pd.read_csv("trial_mapped_pubmed_total_24thMay.csv")
print(pmid_inter_data.shape, "  ----  ", nct_pmid_map_data.shape)

(1130131, 3)   ----   (288632, 3)


In [3]:
print(pmid_inter_data.columns, "  ----  ", nct_pmid_map_data.columns)

Index(['pmid', 'ref_pmid', 'nct_id'], dtype='object')   ----   Index(['nct_id', 'pmid', 'reference_type'], dtype='object')


In [5]:
unique_ncts = list(set(nct_pmid_map_data['nct_id']))

In [6]:
print(len(unique_ncts))

68338


In [6]:
# Forming the first hop neighbors of a PubMed id. It consists of one-to-one mapping along with references and result references made by the clinical trial
# Currently, we are only storing links and not the relation type
first_hop_ncts = defaultdict(list)

for row_id in tqdm(range(nct_pmid_map_data.shape[0])):
    line = nct_pmid_map_data.iloc[row_id, 0]
    line = line.replace('\r', '')
    line = line.replace('\n', '')
    first_hop_ncts[nct_pmid_map_data.iloc[row_id, 1]].append(line)

100%|███████████████████████████████████████████████████████████████████████| 288632/288632 [00:15<00:00, 18462.54it/s]


In [14]:
228604/288632

0.7920258322015576

In [8]:
for key in tqdm(first_hop_ncts.keys()):
    first_hop_ncts[key] = list(set(first_hop_ncts[key]))

100%|███████████████████████████████████████████████████████████████████████| 228604/228604 [00:09<00:00, 24882.39it/s]


In [10]:
# Exporting the defaultdict as a pickle flle
pkl.dump(first_hop_ncts, open('FirstHopNctNeighbors_24thMay.p', 'wb'))

In [11]:
# Working on second-hop neighbors
pmid_inter_data.head()

,pmid,ref_pmid,nct_id
0,6322,13493614,NCT00117845
1,6322,4770788,NCT00117845
2,6322,13387693,NCT00117845
3,6322,13628564,NCT00117845
4,6322,14370191,NCT00117845


In [12]:
second_hop_ncts = defaultdict(list)

for row_id in tqdm(range(pmid_inter_data.shape[0])):
    line = pmid_inter_data.iloc[row_id, 2]
    line = line.replace('\r', '')
    line = line.replace('\n', '')
    second_hop_ncts[pmid_inter_data.iloc[row_id, 1]].append(line)
    
for key in tqdm(second_hop_ncts.keys()):
    second_hop_ncts[key] = list(set(second_hop_ncts[key]))

100%|██████████████████████████████████████████████████████████████████████| 543418/543418 [00:00<00:00, 952948.49it/s]


In [14]:
# Exporting the defaultdict as a pickle flle
pkl.dump(second_hop_ncts, open('SecondHopNctNeighbors_24thMay.p', 'wb'))